In [1]:
import os 
from apikey import apikey 


os.environ['OPENAI_API_KEY'] = apikey

In [2]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
text = "what would be a good company name for a company that makes colorful socks?"
print(llm(text))



Cheerful Socks Co.


## prompt template

In [5]:
data_info = f"""
dataset name: 行业市场变化_市场大盘分析_行业市场大盘_魔镜

dataset description:
primary keys:[月份, 类目编码]
3540 rows
| field | type | uniques | nan_percentage | max | min |
| --- | --- | --- | --- | --- | --- |
| 月份 | DateTime | 59 | 0.0 | nan | nan |
| 平台编码 | String | 2 | 0.0 | nan | nan |
| 一级类目 | String | 4 | 0.0 | nan | nan |
| 二级类目 | String | 8 | 6.666666666666667 | nan | nan |
| 三级类目 | String | 16 | 20.0 | nan | nan |
| 四级类目 | String | 32 | 46.666666666666664 | nan | nan |
| 类目名称 | String | 60 | 0.0 | nan | nan |
| 类目编码 | Numeric | 60 | 0.0 | 39264.0 | 10209.0 |
| 销售额 | Numeric | 3540 | 0.0 | 99989.30547 | 1.8873 |
| 销量 | Numeric | 974 | 0.0 | 999.0 | 0.0 |
| 店铺数 | Numeric | 181 | 0.0 | 200.0 | 20.0 |
| 商品数 | Numeric | 451 | 0.0 | 500.0 | 50.0 |
| 收藏次数 | Numeric | 1 | 0.0 | 0.0 | 0.0 |
"""

In [8]:
from langchain import PromptTemplate

template = """
    Question: {question}
    你是一个数据分析机器人，用户会向你咨询关于数据的问题，请按照以下步骤进行处理。
        1. 需要判断用户的问题是否为这两类【常规数据咨询，BA看板】，对于这两个问题之外的，请礼貌的回答无法满足要求。
            想知道具体单一数值或简单问题的，为数据咨询；若用户想知道多个数据的占比或趋势或涉及多个数据维度之间的关系，属于BA看板。
        2A. 如果为数据咨询，假设dataframe的变量名为df, 写一段可运行的<python代码>，运行后可以打印出回答此问题的信息。
        2B. 如果为BA看板，假设dataframe的变量名为df, 写一段可运行的<python代码>，运行后可以绘制对应需求。
        3. 以JSON格式返回信息。Ensure the response can be parsed by Python json.loads。Json外不能有任何字符
        "message":<所有需要沟通的内容>, "task":<问题分类>, "python_code":<python code>
        
        要求：写python代码时需假设df已经读取在内存中，你不需要写读取数据的代码。

        你掌握的数据情况如下：{data_info}

"""

prompt = PromptTemplate(template=template, input_variables=['question', 'data_info'])

prompt.format(question = "目前销量最好的品类是哪个？", data_info=data_info)


'\n    Question: 目前销量最好的品类是哪个？\n    你是一个数据分析机器人，用户会向你咨询关于数据的问题，请按照以下步骤进行处理。\n        1. 需要判断用户的问题是否为这两类【常规数据咨询，BA看板】，对于这两个问题之外的，请礼貌的回答无法满足要求。\n            想知道具体单一数值或简单问题的，为数据咨询；若用户想知道多个数据的占比或趋势或涉及多个数据维度之间的关系，属于BA看板。\n        2A. 如果为数据咨询，假设dataframe的变量名为df, 写一段可运行的<python代码>，运行后可以打印出回答此问题的信息。\n        2B. 如果为BA看板，假设dataframe的变量名为df, 写一段可运行的<python代码>，运行后可以绘制对应需求。\n        3. 以JSON格式返回信息。Ensure the response can be parsed by Python json.loads。Json外不能有任何字符\n        "message":<所有需要沟通的内容>, "task":<问题分类>, "python_code":<python code>\n        \n        要求：写python代码时需假设df已经读取在内存中，你不需要写读取数据的代码。\n\n        你掌握的数据情况如下：\ndataset name: 行业市场变化_市场大盘分析_行业市场大盘_魔镜\n\ndataset description:\nprimary keys:[月份, 类目编码]\n3540 rows\n| field | type | uniques | nan_percentage | max | min |\n| --- | --- | --- | --- | --- | --- |\n| 月份 | DateTime | 59 | 0.0 | nan | nan |\n| 平台编码 | String | 2 | 0.0 | nan | nan |\n| 一级类目 | String | 4 | 0.0 | nan | nan |\n| 二级类目 | String | 8 | 6.666666666666667 | nan | nan |\n| 三级类目 | S

In [12]:
llm(prompt) # 这里就不能这么用了，可以用后面的Chain

## Chain

In [11]:
from langchain import LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "目前销量最好的品类是哪个？"
print(llm_chain.run({'question':question, 'data_info':data_info}))



回答：
"message": "以当前数据，目前销量最好的品类是X", "task": "数据咨询", "python_code": "print('以当前数据，目前销量最好的品类是{}'.format(df.loc[df.销量.idxmax(),'类目名称']))"


## Agents and Tools 

In [17]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import Tool

from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI

In [18]:

agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

2



'2\n'

In [ ]:
# TODO 思考下我们需要什么Tools？

In [ ]:
# 怎么把python 编译器工具加进来？

In [ ]:
# You can create the tool to pass to an agent
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run
)

## Memory

In [20]:
from langchain import OpenAI, ConversationChain

# llm = OpenAI(temperature=0)

conversation = ConversationChain(llm = llm,prompt=prompt, verbose=True)

conversation.predict(input={'question':question, 'data_info':data_info})



ValidationError: 1 validation error for ConversationChain
__root__
  Got unexpected prompt input variables. The prompt expects ['question', 'data_info'], but got ['history'] as inputs from memory, and input as the normal input key. (type=value_error)